In [77]:
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import pandas as pd
import autosklearn.classification
import os
import featuretools as ft
import warnings
warnings.filterwarnings('ignore')

In [96]:
def run_as(X, y, target_ft, time_budget=30, include_preprocessors = None):
    try:
        os.remove('/tmp/autosklearn_regression_example_tmp')
        os.remove('/tmp/autosklearn_regression_example_out')
    except:
        pass
    #X = df.drop(columns=target_ft)
    #y = df[target_ft]
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)
    automl = autosklearn.classification.AutoSklearnClassifier(
        time_left_for_this_task=time_budget,
        per_run_time_limit=30,
        tmp_folder='./tmp/autosklearn_regression_example_tmp',
        output_folder='./tmp/autosklearn_regression_example_out',
        include_preprocessors = include_preprocessors
    )
    automl.fit(X_train, y_train)
    y_hat = automl.predict(X_test)
    return "Accuracy score - " + str(sklearn.metrics.accuracy_score(y_test, y_hat))
    
    
def gen_feats(df):
    es = ft.EntitySet(id = 'df')
    es.entity_from_dataframe(entity_id = 'data', dataframe = df, 
                         make_index = True, index = 'index')
    feature_matrix, feature_defs = ft.dfs(entityset = es, target_entity = 'data',
                                      agg_primitives=["mean", "max", "min", "std", "skew"],
                                      trans_primitives = ['add_numeric', 'multiply_numeric'])
    return feature_matrix

def run_test(df,target_ft, mode = 0, time_budget=30):
    results = []
    X = df.drop(columns=target_ft)
    y = df[target_ft]
    if mode!= 1:
        rs = run_as(X,y,target_ft, include_preprocessors=None )   
        results.append("Autosk Only with Preprocessing: " + rs)
    if mode!=0:
        rs = run_as(X,y,target_ft, include_preprocessors =["no_preprocessing"])
        results.append("Autosk Only without Preprocessing: " + rs)
    if mode!=0:
        X = gen_feats(X)
        rs = run_as(X,y,target_ft, include_preprocessors =["no_preprocessing"])
        results.append("Autosk with Featuretools: " + rs)    
    
    
    print("===================================")
    [print(x) for x in results]

In [93]:
df = pd.read_csv("blood.csv")
target_ft = "class"
run_test(df, target_ft, mode=2, time_budget=60)

[WARNING] [2020-07-13 10:40:29,328:AutoML(1):828fa7cf24a4202d3c29fc1fbc4b82b7] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (29.604836)
[WARNING] [2020-07-13 10:40:29,331:AutoML(1):828fa7cf24a4202d3c29fc1fbc4b82b7] Capping the per_run_time_limit to 14.0 to have time for a least 2 models in each process.
[WARNING] [2020-07-13 10:40:33,368:EnsembleBuilder(1):828fa7cf24a4202d3c29fc1fbc4b82b7] No models better than random - using Dummy Score!Number of models besides current dummy model: 2. Number of dummy models: 1
[WARNING] [2020-07-13 10:40:35,384:EnsembleBuilder(1):828fa7cf24a4202d3c29fc1fbc4b82b7] No models better than random - using Dummy Score!Number of models besides current dummy model: 5. Number of dummy models: 1
[WARNING] [2020-07-13 10:40:37,401:EnsembleBuilder(1):828fa7cf24a4202d3c29fc1fbc4b82b7] No models better than random - using Dummy Score!Number of models besides current dummy model: 8. Nu

In [94]:
df = pd.read_csv("winequality-red.csv")
target_ft = "quality"
run_test(df, target_ft, mode=2 ,time_budget=60)

[WARNING] [2020-07-13 10:41:45,030:AutoML(1):e2533b21a92f20edcc0be10c40ace137] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (29.599561)
[WARNING] [2020-07-13 10:41:45,034:AutoML(1):e2533b21a92f20edcc0be10c40ace137] Capping the per_run_time_limit to 14.0 to have time for a least 2 models in each process.
[WARNING] [2020-07-13 10:42:10,545:AutoML(1):e2533b21a92f20edcc0be10c40ace137] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (29.741546)
[WARNING] [2020-07-13 10:42:10,548:AutoML(1):e2533b21a92f20edcc0be10c40ace137] Capping the per_run_time_limit to 14.0 to have time for a least 2 models in each process.
[WARNING] [2020-07-13 10:42:38,141:AutoML(1):7f055245b5a1b700be3b9f7176e7bc82] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (29.642647)
[WARN

In [97]:
df = pd.read_csv("blood.csv")
target_ft = "class"
df = gen_feats(df.drop(columns=target_ft))

In [98]:
df

V1  V2     V3  V4  V1 + V3  V2 + V4  V3 + V4  V1 + V4  V2 + V3  \
index                                                                   
0       2  50  12500  98    12502      148    12598      100    12550   
1       0  13   3250  28     3250       41     3278       28     3263   
2       1  16   4000  35     4001       51     4035       36     4016   
3       2  20   5000  45     5002       65     5045       47     5020   
4       1  24   6000  77     6001      101     6077       78     6024   
...    ..  ..    ...  ..      ...      ...      ...      ...      ...   
743    23   2    500  38      523       40      538       61      502   
744    21   2    500  52      521       54      552       73      502   
745    23   3    750  62      773       65      812       85      753   
746    39   1    250  39      289       40      289       78      251   
747    72   1    250  72      322       73      322      144      251   

       V1 + V2  ...  V1 * V2 + V4  V1 + V4 * V4  V2 * V2 + V3  V1 + V3 * V4  \
index           ...                                                           
0           52  ...           296          9800        627500       1225196   
1           13  ...             0           784         42419         91000   
2           17  ...            51          1260         64256        140035   
3           22  ...           130          2115        100400        225090   
4           25  ...           101          6006        144576        462077   
...        ...  ...           ...           ...           ...           ...   
743         25  ...           920          2318          1004         19874   
744         23  ...          1134          3796          1004         27092   
745         26  ...          1495          5270          2259         47926   
746         40  ...          1560          3042           251         11271   
747         73  ...          5256         10368           251         23184   

       V1 + V3 * V3 + V4  V2 + V4 * V3  V1 + V4 * V3 + V4  V2 + V3 * V3  \
index                                                                     
0              157500196       1850000            1259800     156875000   
1               10653500        133250              91784      10604750   
2               16144035        204000             145260      16064000   
3               25235090        325000             237115      25100000   
4               36468077        606000             474006      36144000   
...                  ...           ...                ...           ...   
743               281374         20000              32818        251000   
744               287592         27000              40296        251000   
745               627676         48750              69020        564750   
746                83521         10000              22542         62750   
747               103684         18250              46368         62750   

       V1 + V4 * V3  V1 * V2 + V3  
index                              
0           1250000         25100  
1             91000             0  
2            144000          4016  
3            235000         10040  
4            468000          6024  
...             ...           ...  
743           30500         11546  
744           36500         10542  
745           63750         17319  
746           19500          9789  
747           36000         18072  

[748 rows x 55 columns]

In [90]:
!rm -r tmp